![ETL](<../5_Sources/Images/banner_etl.gif>)

<p align="center">

## **Unificación de Datasets extraidos**

</p>

A través de la API se realiza extracción de multiples datasets en `JSON` por restricciones de la plataforma **HOTELBEDS** la cual permite en el caso de datos de HOTELES extraer por País y por lotes de 1000, su funcionalidad se limita a 50 extracciones cada 24H lo que nos indica que si tenemos aproximadamente **40Mil** Hoteles lo podemos hacer con una cuenta de API limitado a 40 lotes (40 extracciones) de archivos `JSON` para luego proceder a unificar y enviar directamente a realizarle `EDA`.

<mark>Parte de este script será reutilizado para agregar a la automatización</mark>

In [1]:
# Usaremos librería Pandas y Json para permitir la lectura de los archivos
import pandas as pd
import json

### Alistamos variables antes de realizar la iteración con `for`

In [2]:
# Inicializa un DataFrame vacío
df_hotels = pd.DataFrame()
# Número total de conjuntos de datos que deseas leer
num_datasets = 40
# Ruta base donde se encuentran los archivos JSON
path_base = "../2_Datasets/original/hotelbeds/"

### Creamos la iteración con `for` concatenando al mismo tiempo los datasets en JSON ya que tienen la misma estructura con `extend()`
Debemos ejecutarlo con la librería JSON ya que nuestra data es tan compleja que haciendo pruebas con archivos .csv se generan errores en nuestro script; esto debido a que su estructura no está diseñada para contener en sus columnas datos anidados como diccionarios, listas de diccionarios, JSON, listas, etc. Debemos tener en cuenta que estos archivos JSON están estructurados en un mix de de infromación que no manejan la misma estructura entonces debemos extraer la información de `hotels`

In [3]:
# Lista para almacenar los datos de los 40 archivos JSON
all_data = []

# Itera sobre los nombres de archivo y los carga en la lista
for i in range(1, num_datasets + 1):
    filename = f"hotels_us_total_api_{i}.json"
    
    try:
        with open(path_base + filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_data.extend(data)  # Agrega los datos al final de la lista
    except FileNotFoundError:
        print(f"El archivo {filename} no fue encontrado.")

In [4]:
# Convierte el formato JSON a DataFrame
df_hotels = pd.DataFrame(all_data)

In [6]:
df_hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39963 entries, 0 to 39962
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   39963 non-null  int64 
 1   name                   39963 non-null  object
 2   description            39923 non-null  object
 3   countryCode            39963 non-null  object
 4   stateCode              39963 non-null  object
 5   destinationCode        39963 non-null  object
 6   zoneCode               39963 non-null  int64 
 7   coordinates            39963 non-null  object
 8   categoryCode           39963 non-null  object
 9   categoryGroupCode      39876 non-null  object
 10  chainCode              33551 non-null  object
 11  accommodationTypeCode  39963 non-null  object
 12  boardCodes             38280 non-null  object
 13  segmentCodes           39166 non-null  object
 14  address                39963 non-null  object
 15  postalCode         

**`Importante`**: para la automatización y la extracción posterior debemos tener en cuenta la columna **`"code"`** con esta vamos a excluir en extracciones futuras datos repetidos, también nos ayuda a extraer los detalles de cada hotel y su disponibilidad de habitaciones en tiempo real.

In [7]:
# Guardamos un archivo csv su funcionalidad no va a ser a modo de dataset sino una forma de excluir data repetitiva en procesos automáticos
df_hotels['code'].to_csv(path_base + "hotels_code.csv")

### Guardamos la unificación del dataset para el proceso de EDA

In [8]:
# Convierte el DataFrame en formato JSON con los parámetros de formato deseados
json_hotels = df_hotels.to_json(orient='records', 
                                date_format='iso', 
                                force_ascii=False)

In [9]:
# Guarda todos los datos en un solo archivo JSON
with open(path_base + "hotels_dataset.json", 'w', encoding='utf-8') as combined_file:
    json.dump(json_hotels, combined_file, separators=(',', ':'), ensure_ascii=False, indent=0)

print("Datos combinados y guardados en 'hotels_dataset.json'")

Datos combinados y guardados en 'hotels_us_total_combined.json'
